TF Lite model with no quantization

In [ ]:
# prompt: load the model at /content/epochs_50_no_sh.h5
import tensorflow as tf
from keras.models import load_model

model = load_model('/content/epochs_50_no_sh.h5')

# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# save the model
open("tflite_model_no_quantization.tflite", "wb").write(tflite_model)

456768

The same model with float 16 quantization

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Set the optimization mode
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Set float16 is the supported type on the target platform
converter.target_spec.supported_types = [tf.float16]

# Convert and Save the model
tflite_model = converter.convert()
open("converted_model_float16.tflite", "wb").write(tflite_model)

235932

Dynamic range quantization

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Set the optimization mode
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Convert and Save the model
tflite_model = converter.convert()
open("converted_model_drq.tflite", "wb").write(tflite_model)

122848

Integer quantization

In [ ]:
# prompt: load all the images sequentially in /content/Jack_BSN_8r_4s.zip
import zipfile
from PIL import Image
import os
path = "/content/Jack_BSN_8r_4s.zip"
with zipfile.ZipFile(path, 'r') as zip_ref:
  zip_ref.extractall('/content/')

In [ ]:
# prompt: take first 100 images as np arrays in Jack_BSN_8r_4s folder and load them into a tensorflow tensor. Make sure that the file names ending with 000.png through 099.png are being read

images = []
for i in range(100):
  image = Image.open('/content/Jack_BSN_8r_4s/image_1_{0:03d}.png'.format(i))
  im_arr = np.array(image)
  im_arr32 = im_arr.astype(np.float32)
  images.append(np.array(im_arr32))


images = np.array(images)
print(images.shape)
train_images = images.reshape((100, 640, 640, 1))


(100, 640, 640)


In [ ]:
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(train_images).batch(1).take(100):
    # Model has only one input so each data point has one element.
    yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Set the optimization mode
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Pass representative dataset to the converter
converter.representative_dataset = representative_data_gen

# Restricting supported target op specification to INT8
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

# Set the input and output tensors to uint8
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

# Convert and Save the model
tflite_model = converter.convert()
open("converted_model_int8.tflite", "wb").write(tflite_model)

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:947: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


124720